In [1]:
from src.DataLoader import gdfLoader,ReceiverLoader,RSSIDataLoader
from func.utils.getMarkerDictList import getMarkerDictList
from src.DataLoader import Room,Receiver

INFO - Logger configured


In [2]:
#Map Data
MapDataPath="data/cartePalaiseau.csv"
map_gdf=gdfLoader(MapDataPath)
#Receiver Data
ReceiverDataPath="data/carteModulePalaiseau.csv"
receiver_gdf=ReceiverLoader(ReceiverDataPath)

In [3]:
receiver_gdf

,macModule,geometry,coordinatesEPSG3857
0,A8032A31204E,POINT (2.20079 48.71316),POINT (244990.444 6226329.680)
1,C45BBE39F42A,POINT (2.20110 48.71315),POINT (245025.333 6226329.187)
2,C45BBE39F9AE,POINT (2.20120 48.71314),POINT (245035.947 6226326.828)
3,A8032A311FAA,POINT (2.20072 48.71317),POINT (244983.036 6226331.224)
4,C45BBE39F56A,POINT (2.20117 48.71318),POINT (245033.645 6226333.296)
5,C45BBE37B346,POINT (2.20087 48.71315),POINT (244999.412 6226328.251)
6,C45BBE386952,POINT (2.20098 48.71316),POINT (245011.849 6226330.207)
7,C45BBE386362,POINT (2.20094 48.71314),POINT (245007.682 6226326.760)
8,C45BBE384CD2,POINT (2.20093 48.71317),POINT (245006.234 6226331.144)
9,C45BBE39870E,POINT (2.20106 48.71315),POINT (245020.561 6226328.967)


In [4]:
from src.DataLoader import room_dict_generator,room_r_tree_generator
room_dict=room_dict_generator(map_gdf)
room_r_tree=room_r_tree_generator(room_dict)


In [5]:
from src.DataLoader import build_receiver_dict
receiver_dict= build_receiver_dict(receiver_gdf,room_dict,room_r_tree)


In [6]:
Threshold1=-45
Threshold2=-70

In [7]:
from src.DataLoader import Receiver
from shapely.geometry import Point,Polygon
from src.utils.geometry import discrete_circle,intersection
import rtree

#need in the class: receiver_dict
def zone_selection(receiver:Receiver,rssi:int,room_r_tree:rtree,room_dict:dict)->Polygon:
    """
    Return the predicted zone for the selected receiver receiving the rssi 
    
    Parameters
    ---
    Input:
    receiver: Receiver
    rssi: int
    
    Output:
    Polygon
    """
    if rssi>Threshold1:
        #return the room in wich the receiver is
        return [receiver.room.cartesian_polygon]
    
    elif Threshold2< rssi < Threshold1:
        #return all the room in wich the receiver is
        point=receiver.cartesian_point
        area=discrete_circle(6,[point.x,point.y],2.5)#create an hexagon with 6 points centered on the receiver
        rooms=list(intersection(area,room_r_tree,room_dict))
        return rooms
    elif rssi<Threshold2:
        point=receiver.cartesian_point
        area=discrete_circle(6,[point.x,point.y],5)#create an hexagon with 6 points centered on the receiver
        rooms=list(intersection(area,room_r_tree,room_dict))
        return rooms    

In [8]:
from shapely import equals_exact
def test_case1(receiver_dict):
    receiver=receiver_dict['A8032A31204A']
    rssi=-40
    zone=zone_selection(receiver,rssi,room_r_tree,room_dict)
    assert len(zone)==1,f'Error: point in more than one room'
    zone=zone[0]
    assert equals_exact(zone,receiver.room.cartesian_polygon),f'Error: {zone} should be {receiver.room.cartesian_polygon}'
test_case1(receiver_dict)
def test_full_treshold(receiver_dict):
    for receiver in receiver_dict.values():
        rssi=-40
        zone=zone_selection(receiver,rssi,room_r_tree,room_dict)
        assert len(zone)==1,f'Error: point in more than one room'
        zone=zone[0]
        assert equals_exact(zone,receiver.room.cartesian_polygon),f'Error: {zone} should be {receiver.room.cartesian_polygon}'
test_full_treshold(receiver_dict)

In [9]:
from src.visualization import MapCreation

map_object=MapCreation(receiver_dict,room_dict)


In [10]:
map_object

In [11]:
import ast
import os
from src.DataLoader import gdfLoader,ReceiverLoader
from src import API
from src.preprocessing import Preprocessor
from src.preprocessing import remove_duplicates_Cleaner,moving_average_Filter,moving_max_Filter,moving_max_averaged_Filter,blank_Filter
import pandas as pd
from src.visualization import rssi_viewer

#API call
callApi=API()
start="2023-07-27 12:04:00.000000"
end="2023-07-27 12:06:10.000000"
mac_module=ast.literal_eval(os.environ['MAC_WEAR_DICT'])['3480']


raw_data=callApi.getRawDataForCartoWear(mac_module,pd.to_datetime(start),pd.to_datetime(end))
#Preprocessing
sampling_time=1
window='10S'
preprocessor=Preprocessor(raw_data,sampling_time=sampling_time)
rssi_df=preprocessor.set_cleaner(remove_duplicates_Cleaner())\
        .set_filter(moving_max_Filter("2s"))\
        .process()
fig=rssi_viewer(rssi_df)


INFO - data Cleaning


In [12]:
rssi_df.dropna(inplace=True)
rssi_df

,timestamp,macModule,rssi
1,2023-07-27 10:04:01.500,A8032A311DAA,-75.0
2,2023-07-27 10:04:01.500,A8032A311F96,-79.0
3,2023-07-27 10:04:02.500,A8032A311DAA,-75.0
4,2023-07-27 10:04:02.500,A8032A311F96,-79.0
5,2023-07-27 10:04:03.000,A8032A311F6A,-72.0
...,...,...,...
474,2023-07-27 10:06:09.000,A8032A311F6A,-57.0
475,2023-07-27 10:06:09.500,A8032A311DAA,-72.0
476,2023-07-27 10:06:09.500,A8032A311F96,-73.0
477,2023-07-27 10:06:10.000,A8032A311F6A,-57.0


In [13]:
fig=rssi_viewer(rssi_df)


In [14]:
import shapely

def add_element_to_list(list_dict,list_name, element):
    if list_name in list_dict:
        list_dict[list_name].append(element)
    else:
        list_dict[list_name] = [element]
def possible_room_union(zone_list:list)->shapely.geometry.Polygon:
    cartesian_polygon_list=[]
    for zone in zone_list:
        cartesian_polygon_list.append(zone)
    return(shapely.union_all(cartesian_polygon_list))
    

In [15]:
#going through all the data and finding the possible rooms for each timestamp and for each receiver
all_zone={}
for row in rssi_df.iterrows():
    #init
    row=row[1]
    timestamp=row['timestamp']
    #all the possible room according to one receiver
    zone_list=zone_selection(receiver_dict[row['macModule']],row['rssi'],room_r_tree,room_dict)
    #union of all the possible room according to one of the receiver
    possible_rooms=possible_room_union(zone_list)
    
    #adding this union to the dict with keys: timestamp
    add_element_to_list(all_zone,timestamp,possible_rooms)
res={}


for timestamp,instant_zone in all_zone.items():
    res[timestamp]=shapely.intersection_all(instant_zone)
res


{Timestamp('2023-07-27 10:04:01.500000'): <GEOMETRYCOLLECTION (POLYGON ((244985.272 6226333.978, 245009.937 6226329.85...>,
 Timestamp('2023-07-27 10:04:02.500000'): <GEOMETRYCOLLECTION (POLYGON ((244985.272 6226333.978, 245009.937 6226329.85...>,
 Timestamp('2023-07-27 10:04:03'): <POLYGON ((244969.199 6226334.021, 244969.628 6226336.593, 244975.663 622633...>,
 Timestamp('2023-07-27 10:04:03.500000'): <GEOMETRYCOLLECTION (POLYGON ((244985.272 6226333.978, 245009.937 6226329.85...>,
 Timestamp('2023-07-27 10:04:04'): <GEOMETRYCOLLECTION (POLYGON ((244969.628 6226336.593, 244973.181 6226335.99...>,
 Timestamp('2023-07-27 10:04:05'): <GEOMETRYCOLLECTION (POLYGON ((244969.628 6226336.593, 244973.181 6226335.99...>,
 Timestamp('2023-07-27 10:04:06'): <POLYGON ((244983.885 6226324.836, 244979.855 6226325.396, 244980.965 622633...>,
 Timestamp('2023-07-27 10:04:09'): <POLYGON ((244969.199 6226334.021, 244969.628 6226336.593, 244975.663 622633...>,
 Timestamp('2023-07-27 10:04:09.500000'): <

transfer the coordinates of the polygon obtained at each timestamp to lat long for plotting on the map

In [16]:
from pyproj import Transformer
transformer = Transformer.from_crs("EPSG:3857", "EPSG:4326")
def polygon_to_lat_lon(polygon:shapely.geometry.Polygon)->shapely.geometry.Polygon:
    """
    Return the room in which the point is located.
    
    """
    coords=polygon.exterior.coords
    point_list=[transformer.transform(x,y) for x,y in coords] 
    polygon=shapely.geometry.Polygon(point_list)
    return(polygon)
res_in_lat_long={}
for timestamp,polygon in res.items():
    if isinstance(polygon,shapely.geometry.MultiPolygon):
        polygon_list=[]
        for poly in polygon.geoms:
            # on enleve les elements qui ne sont pas des polygones (line string,multi line string,point)
            if isinstance(poly, shapely.geometry.Polygon):
                polygon_list.append(polygon_to_lat_lon(poly))
        if polygon_list:
            res_in_lat_long[timestamp]=shapely.geometry.MultiPolygon(polygon_list)
        
    elif isinstance(polygon,shapely.geometry.Polygon) :
        res_in_lat_long[timestamp]=polygon_to_lat_lon(polygon)
res_in_lat_long

{Timestamp('2023-07-27 10:04:03'): <POLYGON ((48.713 2.201, 48.713 2.201, 48.713 2.201, 48.713 2.201, 48.713 2....>,
 Timestamp('2023-07-27 10:04:06'): <POLYGON ((48.713 2.201, 48.713 2.201, 48.713 2.201, 48.713 2.201, 48.713 2....>,
 Timestamp('2023-07-27 10:04:09'): <POLYGON ((48.713 2.201, 48.713 2.201, 48.713 2.201, 48.713 2.201, 48.713 2....>,
 Timestamp('2023-07-27 10:04:09.500000'): <POLYGON ((48.713 2.201, 48.713 2.201, 48.713 2.201, 48.713 2.201, 48.713 2....>,
 Timestamp('2023-07-27 10:04:10'): <POLYGON ((48.713 2.201, 48.713 2.201, 48.713 2.201, 48.713 2.201, 48.713 2....>,
 Timestamp('2023-07-27 10:04:10.500000'): <POLYGON ((48.713 2.201, 48.713 2.201, 48.713 2.201, 48.713 2.201, 48.713 2....>,
 Timestamp('2023-07-27 10:04:11'): <POLYGON ((48.713 2.201, 48.713 2.201, 48.713 2.201, 48.713 2.201, 48.713 2....>,
 Timestamp('2023-07-27 10:04:12'): <POLYGON ((48.713 2.201, 48.713 2.201, 48.713 2.201, 48.713 2.201, 48.713 2....>,
 Timestamp('2023-07-27 10:04:13'): <POLYGON ((48.7

In [17]:
from  src.visualization import timed_room_Feature
from folium.plugins import TimestampedGeoJson

polygonal_features=timed_room_Feature(res_in_lat_long)

In [18]:
polygonal_features

[{'type': 'Feature',
  'geometry': {'type': 'Polygon',
   'coordinates': [[[2.2005957596608132, 48.71318347866627],
     [2.200599610805512, 48.713198725240794],
     [2.200653827434416, 48.71319274639826],
     [2.2006315232373, 48.71319520603638],
     [2.200593481425564, 48.71319951054955],
     [2.200600951910019, 48.71322637832987],
     [2.200572788715362, 48.71322991792419],
     [2.200578153133392, 48.71325115548479],
     [2.20064353197813, 48.7132434126252],
     [2.2006787359714504, 48.713239651807235],
     [2.20075249671936, 48.7132321301705],
     [2.2007401436195986, 48.713183227714254],
     [2.200961709022522, 48.71315879415333],
     [2.2009576857089996, 48.713142865954474],
     [2.2007014476568014, 48.71317161913055],
     [2.2006914764642715, 48.71313235776497],
     [2.2006529197096825, 48.71313722471617],
     [2.2005845233798027, 48.71314474636711],
     [2.2005957596608132, 48.71318347866627]]]},
  'properties': {'times': ['2023-07-27 10:04:03.000000']}},
 {'ty

In [19]:
TimestampedGeoJson({"type": "FeatureCollection","features": polygonal_features},
                    period='PT1S',
                    duration='PT1S',
                    transition_time=500,  
                    auto_play=True,date_options='YYYY/MM/DD HH:mm:ss:SSS',
                    ).add_to(map_object)
map_object.save("output/wip.html")